## XGB Models
Initially single xgb model was created, finally 7 LSTM models were created with random dense layers and dropouts and averaged with 6 xgb models with random hyperparameters.

## Importing Packages

In [1]:
import argparse
import functools
from collections import defaultdict

import numpy as np
import pandas as pd
import xgboost as xgb

from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

## Loading Features
Features were created using feature_engineering.ipynb and page_rank.ipynb

In [2]:
df_train = pd.read_csv('data/train.csv')

In [3]:
y_train = df_train['is_duplicate'].values

In [4]:
train_difflib = pd.read_csv('data/train_feature_difflib.csv')
train_embedding = pd.read_csv('data/train_feature_embedding.csv')
train_embedding2 = pd.read_csv('data/train_feature_embedding_2.csv')
train_fuzz = pd.read_csv('data/train_feature_fuzz.csv')
train_len = pd.read_csv('data/train_feature_len.csv')
#train_match = pd.read_csv('train_feature_match.csv')
train_match_2 = pd.read_csv('data/train_feature_match_2.csv')
#train_oof = pd.read_csv('train_feature_oof.csv')
#train_simhash = pd.read_csv('train_feature_simhash.csv')
train_tfidf = pd.read_csv('data/train_feature_tfidf.csv')

train_graph_clique = pd.read_csv('data/train_feature_graph_clique.csv')
train_graph_pagerank = pd.read_csv('data/train_feature_graph_pagerank.csv')
train_graph_freq = pd.read_csv('data/train_feature_graph_question_freq.csv')
train_graph_intersect = pd.read_csv('data/train_feature_graph_intersect.csv')
X_train = pd.concat([
        train_difflib,
        train_embedding,
        train_embedding2,
        train_fuzz,
        train_len,
        #train_match,
        train_match_2,
        #train_oof,
        #train_simhash,
        train_tfidf,
        train_graph_clique,
        train_graph_pagerank,
        train_graph_freq,
        train_graph_intersect,
    ], axis=1)

In [5]:
test_difflib = pd.read_csv('data/test_feature_difflib.csv')
test_embedding = pd.read_csv('data/test_feature_embedding.csv')
test_embedding2 = pd.read_csv('data/test_feature_embedding_2.csv')
test_fuzz = pd.read_csv('data/test_feature_fuzz.csv')
test_len = pd.read_csv('data/test_feature_len.csv')
#test_match = pd.read_csv('test_feature_match.csv')
test_match_2 = pd.read_csv('data/test_feature_match_2.csv')
#test_oof = pd.read_csv('test_feature_oof.csv')
#test_simhash = pd.read_csv('test_feature_simhash.csv')
test_tfidf = pd.read_csv('data/test_feature_tfidf.csv')

test_graph_clique = pd.read_csv('data/test_feature_graph_clique.csv')
test_graph_pagerank = pd.read_csv('data/test_feature_graph_pagerank.csv')
test_graph_freq = pd.read_csv('data/test_feature_graph_question_freq.csv')
test_graph_intersect = pd.read_csv('data/test_feature_graph_intersect.csv')

X_test = pd.concat([
        test_difflib,
        test_embedding,
        test_embedding2,
        test_fuzz,
        test_len,
        #test_match,
        test_match_2,
        #test_oof,
        #test_simhash,
        test_tfidf,
        test_graph_clique,
        test_graph_pagerank,
        test_graph_freq,
        test_graph_intersect,
    ], axis=1)

In [6]:
X_train.shape, X_test.shape

((404290, 77), (2345796, 77))

In [7]:
X_train.shape

(404290, 77)

In [88]:
X_train.to_csv('lxy_train.csv',index=False)

In [ ]:
X_test.to_csv('lxy_train.csv',index=False)

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=4242)

In [94]:
test_cur = pd.read_csv('lxy_test.csv')

In [97]:
test_cur.to_csv('hello.csv', index=False)

In [98]:
now_cur = pd.read_csv('hello.csv')

In [99]:
now_cur.shape

(2345796, 74)

## UpDown Sampling

In [139]:
#UPDownSampling
pos_train = X_train[y_train == 1]
neg_train = X_train[y_train == 0]
X_train = pd.concat((neg_train, pos_train.iloc[:int(0.8*len(pos_train))], neg_train))
y_train = np.array([0] * neg_train.shape[0] + [1] * pos_train.iloc[:int(0.8*len(pos_train))].shape[0] + [0] * neg_train.shape[0])
print(np.mean(y_train))
del pos_train, neg_train

0.18975293212164862


In [140]:
pos_valid = X_valid[y_valid == 1]
neg_valid = X_valid[y_valid == 0]
X_valid = pd.concat((neg_valid, pos_valid.iloc[:int(0.8 * len(pos_valid))], neg_valid))
y_valid = np.array([0] * neg_valid.shape[0] + [1] * pos_valid.iloc[:int(0.8 * len(pos_valid))].shape[0] + [0] * neg_valid.shape[0])
print(np.mean(y_valid))
del pos_valid, neg_valid

0.18923467767545252


## Lots of XG boost

In [10]:
for i in range(3):
    np.random.seed(i+1)
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    et = [.02,.025,.01,.015]
    params['eta'] = np.random.choice(et)
    params['n_jobs'] = 5
    depth = [4,5,6,7]
    params['max_depth'] = np.random.choice(depth)
    sub = [.5,.6,.7,.4]
    params['subsample'] = np.random.choice(sub)
    params['base_score'] = 0.2
    col = [1,.7]
    params['colsample_bytree'] = np.random.choice(col)
    #params['scale_pos_weight'] = 0.36

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_valid = xgb.DMatrix(X_valid, label=y_valid)

    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    print ('training for fold = '+ str(i+1))
    bst = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=50, verbose_eval=50)
    print ('-> predicting')
    d_test = xgb.DMatrix(X_test)
    p_test = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)
    print('saving to csv')
    sub = pd.read_csv('xgb_final.csv')
    sub[str('fold'+ str(i+1))] = p_test
    sub.to_csv('xgb_final.csv', index=False)

training for fold = 1
[0]	train-logloss:0.70918	valid-logloss:0.70796
[50]	train-logloss:0.33497	valid-logloss:0.33727
[100]	train-logloss:0.27479	valid-logloss:0.27899
[150]	train-logloss:0.25728	valid-logloss:0.26268
[200]	train-logloss:0.24903	valid-logloss:0.25553


KeyboardInterrupt: 

## Training single model

In [141]:
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.02
    params['n_jobs'] = 5
    params['max_depth'] = 6
    params['subsample'] = 0.6
    params['base_score'] = 0.2
    #params['scale_pos_weight'] = 0.36

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_valid = xgb.DMatrix(X_valid, label=y_valid)

    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [20]:
bst = xgb.train(params, d_train, 1500, watchlist, early_stopping_rounds=50, verbose_eval=50)
print(log_loss(y_valid, bst.predict(d_valid)))

[0]	train-logloss:0.70918	valid-logloss:0.70796
[50]	train-logloss:0.33497	valid-logloss:0.33727
[100]	train-logloss:0.27479	valid-logloss:0.27899
[150]	train-logloss:0.25728	valid-logloss:0.26268
[200]	train-logloss:0.24903	valid-logloss:0.25553
[250]	train-logloss:0.24304	valid-logloss:0.25124
[300]	train-logloss:0.23830	valid-logloss:0.24824
[350]	train-logloss:0.23421	valid-logloss:0.24601
[400]	train-logloss:0.23054	valid-logloss:0.24408
[450]	train-logloss:0.22729	valid-logloss:0.24246
[500]	train-logloss:0.22470	valid-logloss:0.24118
[550]	train-logloss:0.22240	valid-logloss:0.24025
[600]	train-logloss:0.22010	valid-logloss:0.23937
[650]	train-logloss:0.21808	valid-logloss:0.23876
[700]	train-logloss:0.21596	valid-logloss:0.23811
[750]	train-logloss:0.21385	valid-logloss:0.23751
[800]	train-logloss:0.21200	valid-logloss:0.23710
[850]	train-logloss:0.20999	valid-logloss:0.23649
[900]	train-logloss:0.20824	valid-logloss:0.23609
[950]	train-logloss:0.20643	valid-logloss:0.23575
[10

In [13]:
xgb

<module 'xgboost' from '/Users/lianxinyu/opt/anaconda3/envs/ml/lib/python3.7/site-packages/xgboost/__init__.py'>

In [14]:
import joblib

In [21]:

#save model
joblib.dump(xgb, 'xgb.model') 

TypeError: can't pickle module objects

## Test

In [ ]:
['diff_ratios', 'google_news_wmd', 'google_news_norm_wmd', 'google_news_cosine_distance', 'google_news_cityblock_distance', 'google_news_jaccard_distance', 'google_news_canberra_distance', 'google_news_euclidean_distance', 'google_news_minkowski_distance', 'google_news_braycurtis_distance', 'google_news_skew_q1vec', 'google_news_skew_q2vec', 'google_news_kur_q1vec', 'google_news_kur_q2vec', 'fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio', 'fuzz_token_sort_ratio', 'char_len1', 'char_len2', 'word_len1', 'word_len2', 'char_len_diff_ratio', 'word_len_diff_ratio', 'word_match_ratio_2', 'word_match_ratio_2_root', 'tfidf_word_match_ratio_2', 'shared_count_2', 'stops1_ratio', 'stops2_ratio', 'shared_2gram', 'word_match_cosine', 'words_hamming', 'diff_stops_r', 'caps_count_q1', 'caps_count_q2', 'diff_caps', 'avg_world_len1', 'avg_world_len2', 'diff_avg_word', 'q1_how', 'q2_how', 'how_both', 'q1_what', 'q2_what', 'what_both', 'q1_which', 'q2_which', 'which_both', 'q1_who', 'q2_who', 'who_both', 'q1_where', 'q2_where', 'where_both', 'q1_when', 'q2_when', 'when_both', 'q1_why', 'q2_why', 'why_both', 'tfidf_sum1', 'tfidf_sum2', 'tfidf_mean1', 'tfidf_mean2', 'tfidf_len1', 'tfidf_len2', 'graph_clique_feat', 'q1_pr', 'q2_pr', 'q1_freq', 'q2_freq', 'q1_q2_intersect', 'q1_q2_wm_ratio']
['diff_ratios', 'google_news_wmd', 'google_news_norm_wmd', 'google_news_cosine_distance', 'google_news_cityblock_distance', 'google_news_jaccard_distance', 'google_news_canberra_distance', 'google_news_euclidean_distance', 'google_news_minkowski_distance', 'google_news_braycurtis_distance', 'fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio', 'fuzz_token_sort_ratio', 'char_len1', 'char_len2', 'word_len1', 'word_len2', 'char_len_diff_ratio', 'word_len_diff_ratio', 'word_match_ratio_2', 'word_match_ratio_2_root', 'tfidf_word_match_ratio_2', 'shared_count_2', 'stops1_ratio', 'stops2_ratio', 'shared_2gram', 'word_match_cosine', 'words_hamming', 'diff_stops_r', 'caps_count_q1', 'caps_count_q2', 'diff_caps', 'avg_world_len1', 'avg_world_len2', 'diff_avg_word', 'q1_how', 'q2_how', 'how_both', 'q1_what', 'q2_what', 'what_both', 'q1_which', 'q2_which', 'which_both', 'q1_who', 'q2_who', 'who_both', 'q1_where', 'q2_where', 'where_both', 'q1_when', 'q2_when', 'when_both', 'q1_why', 'q2_why', 'why_both', 'tfidf_sum1', 'tfidf_sum2', 'tfidf_mean1', 'tfidf_mean2', 'tfidf_len1', 'tfidf_len2', 'graph_clique_feat', 'q1_pr', 'q2_pr', 'q1_freq', 'q2_freq', 'q1_q2_intersect', 'q1_q2_wm_ratio']

In [143]:
d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)
sub = pd.DataFrame()

In [144]:
test_ids = [i for i in range(2345796)]
sub['test_id'] = test_ids
sub['is_duplicate'] = p_test

In [145]:
sub.head()

,test_id,is_duplicate
0,0,0.000589
1,1,0.124254
2,2,0.183836
3,3,0.001145
4,4,0.121627


In [146]:
sub.to_csv('sub_xgb_6.csv',index = False)

In [27]:
import pickle
file_name = "xgb.pkl"

# save
pickle.dump(xgb, open(file_name, "wb"))

TypeError: can't pickle module objects

In [22]:
xgb

<module 'xgboost' from '/Users/lianxinyu/opt/anaconda3/envs/ml/lib/python3.7/site-packages/xgboost/__init__.py'>

In [25]:
pickle.dump(xgb, open("pima.pickle.dat", "wb"))

TypeError: can't pickle module objects